# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [17]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

In [5]:
# Your code here
df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [10]:
# Your code here
formula = 'len ~ C(supp) + C(dose)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

,sum_sq,df,F,PR(>F)
C(supp),205.350000,1.0,14.016638,4.292793e-04
C(dose),2426.434333,2.0,82.810935,1.871163e-17
Residual,820.425000,56.0,NaN,NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [11]:
# Your comment here
'''
Assuming the alpha value = 0.05, the table shows both the supplement and dosage have a significant impact on the tooth length.
'''

'\nAssuming the alpha value = 0.05, the table shows both the supplement and dosage have a significant impact on the tooth length.\n'

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [12]:
unique_supplements = df['supp'].unique()
unique_supplements

array(['VC', 'OJ'], dtype=object)

In [14]:
# Your code here
oj_data = df.loc[df['supp'] == 'OJ', 'len']
vc_data = df.loc[df['supp'] == 'VC', 'len']

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [19]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
t_value, p_value = stats.ttest_ind(oj_data, vc_data)
p_value

np.float64(0.06039337122412848)

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [20]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
table

,sum_sq,df,F,PR(>F)
C(supp),205.350000,1.0,3.668253,0.060393
Residual,3246.859333,58.0,NaN,NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [22]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', np.float64(0.5))
('OJ', np.float64(1.0))
('OJ', np.float64(2.0))
('VC', np.float64(0.5))
('VC', np.float64(1.0))
('VC', np.float64(2.0))


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [ ]:
len(group_names)

6

In [32]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
grouped = df.groupby(['supp', 'dose'])['len']

group_names = list(grouped.groups.keys())
combinations =[(group1, group_names[j])
              for i, group1 in enumerate(group_names)
              for j in range(i + 1, len(group_names))]

for group1_name, group2_name in combinations:
    group1_data = grouped.get_group(group1_name)
    group2_data = grouped.get_group(group2_name)

    t_stat, p_value = stats.ttest_ind(group1_data, group2_data)
    print(f"Group 1: {group1_name}, Group 2: {group2_name}, P-value: {p_value}")

Group 1: ('OJ', 0.5), Group 2: ('OJ', 1.0), P-value: 8.357559281443774e-05
Group 1: ('OJ', 0.5), Group 2: ('OJ', 2.0), P-value: 3.4018585295016214e-07
Group 1: ('OJ', 0.5), Group 2: ('VC', 0.5), P-value: 0.005303661339923052
Group 1: ('OJ', 0.5), Group 2: ('VC', 1.0), P-value: 0.04223992429368204
Group 1: ('OJ', 0.5), Group 2: ('VC', 2.0), P-value: 7.025409196997986e-06
Group 1: ('OJ', 1.0), Group 2: ('OJ', 2.0), P-value: 0.03736279585664383
Group 1: ('OJ', 1.0), Group 2: ('VC', 0.5), P-value: 1.3372624230559431e-08
Group 1: ('OJ', 1.0), Group 2: ('VC', 1.0), P-value: 0.000780726165177447
Group 1: ('OJ', 1.0), Group 2: ('VC', 2.0), P-value: 0.09583711277517494
Group 1: ('OJ', 2.0), Group 2: ('VC', 0.5), P-value: 1.3381068810881244e-11
Group 1: ('OJ', 2.0), Group 2: ('VC', 1.0), P-value: 2.3131084633597503e-07
Group 1: ('OJ', 2.0), Group 2: ('VC', 2.0), P-value: 0.9637097790041267
Group 1: ('VC', 0.5), Group 2: ('VC', 1.0), P-value: 6.492264598157612e-07
Group 1: ('VC', 0.5), Group 2: (

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.